In [3]:
import os

import pandas              as pd
import numpy               as np
import matplotlib.pyplot   as plt

from keras.layers          import Dense
from keras.models          import Sequential, load_model
from keras.callbacks       import EarlyStopping
from keras.metrics         import categorical_crossentropy
from sklearn.preprocessing import LabelBinarizer, MinMaxScaler

In [5]:
# Creates folders
os.makedirs("../results/", exist_ok=True)
os.makedirs("../models/", exist_ok=True)
os.makedirs("../submission/", exist_ok=True)

In [6]:
# Importing train and test data
train = pd.read_csv('../data/train.zip')
test = pd.read_csv('../data/test.zip')

In [7]:
# Separating the target data from the train data
target = train.label
train.drop(columns= ['label'], axis = 1, inplace = True)

In [8]:
# Formatting the data
normalizer = MinMaxScaler(feature_range = (0, 1))

# Transform data in binary
lb = LabelBinarizer()

# Normalize input train and test data
x_train = normalizer.fit_transform(train)
x_test = normalizer.fit_transform(test)

y_train = lb.fit_transform(target)

In [11]:
def new_model(hidden_layers = 1, nodes = 10, patience = 5):

    file_name = f'no_cnn_{hidden_layers}_layer_{nodes}_nodes'

    # Instantiating model
    model = Sequential()

    # Creating early_stopping
    early_stopping_monitor = EarlyStopping(patience = patience,
                                           monitor='val_loss')

    # Adding hidden layers
    for _ in range (0, hidden_layers):
        
        model.add(Dense(nodes, activation = 'relu'))

    # Adding final layer
    model.add(Dense(10, activation = 'softmax'))

    # Compiling the model
    model.compile(optimizer = 'adam', 
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])

    # Fit the model
    model.fit(x_train, 
              y_train, 
              epochs = 100,  
              shuffle = True, 
              verbose = 2,
              callbacks = [early_stopping_monitor],
              validation_split = 0.3)

    # Creating submission file
    submission = pd.DataFrame(np.argmax(model.predict(x_test), axis = -1), columns = ['Label'])

    # Creating ImageID index
    submission.rename_axis('ImageId', inplace=True)

    #Saving model
    model.save(f'../models/{file_name}.h5')

    # Saving submission file
    submission.to_csv(f'../submission/{file_name}.csv')


In [12]:
# Best result so far!
new_model(hidden_layers = 3, nodes = 40)

Epoch 1/100
919/919 - 3s - loss: 0.4318 - accuracy: 0.8692 - val_loss: 0.2128 - val_accuracy: 0.9368
Epoch 2/100
919/919 - 1s - loss: 0.1912 - accuracy: 0.9433 - val_loss: 0.1999 - val_accuracy: 0.9371
Epoch 3/100
919/919 - 2s - loss: 0.1433 - accuracy: 0.9574 - val_loss: 0.1471 - val_accuracy: 0.9556
Epoch 4/100
919/919 - 1s - loss: 0.1143 - accuracy: 0.9647 - val_loss: 0.1493 - val_accuracy: 0.9538
Epoch 5/100
919/919 - 1s - loss: 0.0971 - accuracy: 0.9701 - val_loss: 0.1270 - val_accuracy: 0.9612
Epoch 6/100
919/919 - 1s - loss: 0.0820 - accuracy: 0.9747 - val_loss: 0.1628 - val_accuracy: 0.9525
Epoch 7/100
919/919 - 1s - loss: 0.0734 - accuracy: 0.9764 - val_loss: 0.1429 - val_accuracy: 0.9589
Epoch 8/100
919/919 - 1s - loss: 0.0627 - accuracy: 0.9803 - val_loss: 0.1315 - val_accuracy: 0.9641
Epoch 9/100
919/919 - 1s - loss: 0.0560 - accuracy: 0.9820 - val_loss: 0.1449 - val_accuracy: 0.9604
Epoch 10/100
919/919 - 1s - loss: 0.0511 - accuracy: 0.9836 - val_loss: 0.1245 - val_accura